This is still a work in progress, so comments are not yet complete.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
df = pd.read_csv("train_values.csv")
df.head()

,row_id,doc_text
0,0,"For more information, visit http://www.wor..."
1,1,...
2,2,...
3,3,71399\n\nProcur...
4,4,90189\n\n\n\n\nExecu...


In [4]:
df.doc_text[0]

'    For more information, visit http://www.worldbank.org/prospects                                                                  98944\n\n    Taking Stock\n   ECB unveiled 1.1 trillion asset-purchase program. The European Central Bank announced it will inject about 1.1\n    trillion ($1.3 trillion) into financial markets through an asset-purchase program in a bid to counter weaker than expected\n    inflation dynamics. Beginning March this year, the ECB will buy a total of 60 billion ($69 billion) a month of public and\n    private sector securities until end-September 2016. These purchases may continue beyond September 2016 unless there\n    are clear signs of a sustained adjustment in the path of infl ation towards ECBs target of close to 2 percent. Following\n    the announcement, European stocks rallied, while the euro weakened to an 11-year low against the dollar.\n\n   World Bank released its January 2015 Commodity Markets Outlook. Prices of most commodities are expected to\n

In [5]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.5)
vectorizer.fit(df["doc_text"])
len(vectorizer.vocabulary_)

37560

In [6]:
train_vector = vectorizer.transform(df["doc_text"])
train_vector

<18830x37560 sparse matrix of type '<class 'numpy.float64'>'
	with 10245656 stored elements in Compressed Sparse Row format>

In [7]:
input_len = train_vector.shape[1]
input_len

37560

In [8]:
X_train = train_vector.toarray()

In [9]:
df = pd.read_csv("train_labels.csv")
df.head()

,row_id,information_and_communication_technologies,governance,urban_development,law_and_development,public_sector_development,agriculture,communities_and_human_settlements,health_and_nutrition_and_population,culture_and_development,...,private_sector_development,informatics,energy,social_development,water_resources,education,transport,water_supply_and_sanitation,gender,infrastructure_economics_and_finance
0,0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y_train = np.array(df.iloc[:,1:])

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_1, X_2, y_1, y_2 = train_test_split(X_train, y_train, test_size=0.05)

In [13]:
model = Sequential()
model.add(Dense(900, activation="relu", input_shape=(input_len,)))
model.add(Dense(29, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data=(X_2, y_2), epochs=9, batch_size=500)

Train on 18830 samples, validate on 942 samples
Epoch 1/9
18830/18830 [==============================] - 73s 4ms/step - loss: 0.4121 - acc: 0.8800 - val_loss: 0.2994 - val_acc: 0.8966
Epoch 2/9
18830/18830 [==============================] - 71s 4ms/step - loss: 0.2754 - acc: 0.9007 - val_loss: 0.2514 - val_acc: 0.9056
Epoch 3/9
18830/18830 [==============================] - 71s 4ms/step - loss: 0.2335 - acc: 0.9121 - val_loss: 0.2166 - val_acc: 0.9172
Epoch 4/9
18830/18830 [==============================] - 70s 4ms/step - loss: 0.2039 - acc: 0.9233 - val_loss: 0.1925 - val_acc: 0.9270
Epoch 5/9
18830/18830 [==============================] - 71s 4ms/step - loss: 0.1830 - acc: 0.9307 - val_loss: 0.1743 - val_acc: 0.9335
Epoch 6/9
18830/18830 [==============================] - 67s 4ms/step - loss: 0.1667 - acc: 0.9364 - val_loss: 0.1591 - val_acc: 0.9384
Epoch 7/9
18830/18830 [==============================] - 119s 6ms/step - loss: 0.1528 - acc: 0.9412 - val_loss: 0.1459 - val_acc: 0.9441

In [14]:
df = pd.read_csv("test_values.csv")
test_vector = vectorizer.transform(df["doc_text"])
X_submit = test_vector.toarray()

In [15]:
from sklearn.metrics import f1_score
y_pred = model.predict(X_2)
for i in range(1, 65):
    y_pred_bin = np.where(y_pred > (i / 100), 1, 0)
    print(i/100, f1_score(y_2, y_pred_bin, average="micro"))

0.01 0.30200156985871274
0.02 0.3698654924905
0.03 0.4281812451852371
0.04 0.4772940808017539
0.05 0.5188420692017814
0.06 0.5565410199556542
0.07 0.5879352623835212
0.08 0.6152733285109021
0.09 0.6385685027487334
0.1 0.6569065713008493
0.11 0.6760009257116408
0.12 0.6938239159001314
0.13 0.7077378018728437
0.14 0.7184074587375584
0.15 0.729708837928369
0.16 0.7381794368041913
0.17 0.7486331510868116
0.18 0.7574028796522684
0.19 0.7637719177136546
0.2 0.7694682194471727
0.21 0.7758128921848259
0.22 0.7820049182699262
0.23 0.7879143443825168
0.24 0.7928061831153389
0.25 0.7967455175531113
0.26 0.79945180447693
0.27 0.8022215365627893
0.28 0.8053042121684867
0.29 0.8058527375707991
0.3 0.8047036389639282
0.31 0.8060995184590689
0.32 0.8063420158550396
0.33 0.8067199478062306
0.34 0.8070406316828426
0.35 0.8079734219269102
0.36 0.808695652173913
0.37 0.8071537033912602
0.38 0.8061883713022782
0.39 0.8023355658595227
0.4 0.8031822898650984
0.41 0.7995817357964448
0.42 0.7958538299367534
0.

In [16]:
y_submit = model.predict(X_submit)

In [17]:
## check cut-off value below
y_submit_bin = np.where(y_submit > 0.31, 1, 0)

In [18]:
df2 = pd.read_csv("submission_format.csv")
df2.head()

,row_id,information_and_communication_technologies,governance,urban_development,law_and_development,public_sector_development,agriculture,communities_and_human_settlements,health_and_nutrition_and_population,culture_and_development,...,private_sector_development,informatics,energy,social_development,water_resources,education,transport,water_supply_and_sanitation,gender,infrastructure_economics_and_finance
0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [19]:
df2.shape

(18831, 30)

In [20]:
y_submit_bin2 = np.column_stack((np.array(df2.row_id).reshape(18831,1), y_submit_bin))

In [21]:
subdf = pd.DataFrame(data=y_submit_bin2, columns=df2.columns, index=None)

In [22]:
subdf.to_csv("submission15.csv", index=False)

In [23]:
#F1 = 0.6611